<a href="https://colab.research.google.com/github/iam-fern/Generate_data/blob/main/ex_sampling_c_100rep_n20_x2_sas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import statistics as stats
import scipy.stats as ss
import scipy.stats as st
from scipy.stats import f_oneway
from scipy import stats
import scipy.stats as stats
import time
import statsmodels.api as sm

from statsmodels.formula.api import ols

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv('https://github.com/iam-fern/Research/raw/main/sampling_100rep_n20_x2_sas.csv', index_col=['rep', 'index'])
df

x1         x2  ...       y_hat        e_hat
rep  index                        ...                         
0.0  0.0    39.356822  34.141232  ...  428.841218  1143.973288
     1.0    22.832811  30.970436  ...  323.626146   711.656047
     2.0    17.550017  20.425372  ...  283.812669    80.849304
     3.0    15.944783  47.292068  ...  207.535023   258.864678
     4.0    33.913074  26.372402  ...  413.042488 -1523.105316
...               ...        ...  ...         ...          ...
99.0 15.0   38.589134  31.844129  ...  417.691494  1861.253771
     16.0   17.802384  31.033780  ...  283.468692   -71.703873
     17.0   12.152475  14.318230  ...  237.823140  -248.927749
     18.0   21.735816  11.444608  ...  339.607336   -31.843668
     19.0   26.998311  42.877700  ...  358.099777  1478.142767

[2000 rows x 7 columns]

In [3]:
# ลบข้อมูลที่มี missing values
table = df.dropna()
table

x1         x2  ...       y_hat        e_hat
rep  index                        ...                         
0.0  0.0    39.356822  34.141232  ...  428.841218  1143.973288
     1.0    22.832811  30.970436  ...  323.626146   711.656047
     2.0    17.550017  20.425372  ...  283.812669    80.849304
     3.0    15.944783  47.292068  ...  207.535023   258.864678
     4.0    33.913074  26.372402  ...  413.042488 -1523.105316
...               ...        ...  ...         ...          ...
99.0 15.0   38.589134  31.844129  ...  417.691494  1861.253771
     16.0   17.802384  31.033780  ...  283.468692   -71.703873
     17.0   12.152475  14.318230  ...  237.823140  -248.927749
     18.0   21.735816  11.444608  ...  339.607336   -31.843668
     19.0   26.998311  42.877700  ...  358.099777  1478.142767

[2000 rows x 7 columns]

In [4]:
table.reset_index(inplace=True)
table

,rep,index,x1,x2,x3,e,Y,y_hat,e_hat
0,0.0,0.0,39.356822,34.141232,39.280579,1100.824471,1572.814505,428.841218,1143.973288
1,0.0,1.0,22.832811,30.970436,19.580178,751.403466,1035.282193,323.626146,711.656047
2,0.0,2.0,17.550017,20.425372,24.844378,138.892047,364.661972,283.812669,80.849304
3,0.0,3.0,15.944783,47.292068,49.758903,204.900902,466.399701,207.535023,258.864678
4,0.0,4.0,33.913074,26.372402,24.351898,-1504.917867,-1110.062827,413.042488,-1523.105316
...,...,...,...,...,...,...,...,...,...
1995,99.0,15.0,38.589134,31.844129,44.452775,1811.757025,2278.945265,417.691494,1861.253771
1996,99.0,16.0,17.802384,31.033780,18.619329,-20.912128,211.764819,283.468692,-71.703873
1997,99.0,17.0,12.152475,14.318230,30.518502,-182.466094,-11.104609,237.823140,-248.927749
1998,99.0,18.0,21.735816,11.444608,16.345481,57.615422,307.763669,339.607336,-31.843668


In [5]:
table.set_index(['rep', 'index'], inplace=True)

In [6]:
table

x1         x2  ...       y_hat        e_hat
rep  index                        ...                         
0.0  0.0    39.356822  34.141232  ...  428.841218  1143.973288
     1.0    22.832811  30.970436  ...  323.626146   711.656047
     2.0    17.550017  20.425372  ...  283.812669    80.849304
     3.0    15.944783  47.292068  ...  207.535023   258.864678
     4.0    33.913074  26.372402  ...  413.042488 -1523.105316
...               ...        ...  ...         ...          ...
99.0 15.0   38.589134  31.844129  ...  417.691494  1861.253771
     16.0   17.802384  31.033780  ...  283.468692   -71.703873
     17.0   12.152475  14.318230  ...  237.823140  -248.927749
     18.0   21.735816  11.444608  ...  339.607336   -31.843668
     19.0   26.998311  42.877700  ...  358.099777  1478.142767

[2000 rows x 7 columns]

In [7]:
def get_result(run_replicate):
    new_table = table.loc[run_replicate]

    # show index
    new_table.reset_index(inplace=True)

    new_table.drop(['index'], axis=1, inplace=True)


    # แบ่งข้อมูลออกเป็น 2 กลุ่มด้วย k-mean

    #Set number of clusters
    x = new_table[["x1", "x2", "x3"]]
    k_means = KMeans(n_clusters=2)

    #Run the clustering algorithm
    model = k_means.fit(x)
    model

    #Generate cluster predictions and store in y_kmeans
    y_kmeans = k_means.predict(x)
    y_kmeans

    # kmean_clusters.astype(float)
    kmean_clusters = pd.DataFrame({"clusters" : y_kmeans})

    # เชื่อมข้อมูลทั้ง 2 DataFrame เข้าด้วยกันด้วยคำสั่ง pd.concat
    data_clusters = pd.concat([new_table, kmean_clusters], axis=1)

    data_sort_clusters = data_clusters.sort_values('clusters')

    # print(data_clusters.groupby('clusters').groups)

    data_sort_clusters.reset_index(inplace=True)

    # กลุ่มที่ 1
    data_cluster1 = data_sort_clusters[data_sort_clusters.clusters == 1]

    y = data_cluster1[['Y']]
    x = data_cluster1[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model1 = sm.OLS(y, x).fit()

    DF1 = model1.df_resid
    SSR1 = model1.ssr
    MSE1 = model1.mse_resid

    # กลุ่มที่ 2
    data_cluster0 = data_sort_clusters[data_sort_clusters.clusters == 0]

    y = data_cluster0[['Y']]
    x = data_cluster0[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model0 = sm.OLS(y, x).fit()

    DF0 = model0.df_resid
    SSR0 = model0.ssr
    MSE0 = model0.mse_resid

    df1 = DF1+1
    df0 = DF0+1

    # คำนวณค่าสถิติ
    GQ_test_clusters = (SSR1/df1) / (SSR0/df0)


    # คำนวณค่า p-value
    # sf(x, dfn, dfd, loc=0, scale=1) : Survival function (also defined as 1 - cdf, but sf is sometimes more accurate)
    two_tailed = 2.0*(1.0-(ss.f.cdf(GQ_test_clusters, df1, df0)))

    # pdf(x, dfn, dfd, loc=0, scale=1) : Probability density function
    p_greater = ss.f.cdf(GQ_test_clusters, df1, df0)
    
    return df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater

# สร้าง list ของ replicates
list_replicate=list({i[0] for i in table.index})
list_replicate.sort()

# สร้าง list เพื่อเก็บค่า ทุกรอบที่ run
list_df0=[]
list_df1=[]
list_SSR0=[]
list_SSR1=[]
list_MSE0=[]
list_MSE1=[]
list_GQ_test_clusters=[]
list_result_test=[]
list_two_tailed=[]
list_p_greater=[]
list_result_test01=[]

# ใช้ for loop สำหรับการ run ทุก replicates
for run_replicate in list_replicate:
    df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater=get_result(run_replicate)

    list_df0.append(df0)
    list_df1.append(df1)
    list_SSR0.append(SSR0)
    list_SSR1.append(SSR1)
    list_MSE0.append(MSE0)
    list_MSE1.append(MSE1)
    list_GQ_test_clusters.append(GQ_test_clusters)
    list_two_tailed.append(two_tailed)
    list_p_greater.append(p_greater)

/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.py:1665: RuntimeWarning: divide by zero encountered in double_scalars
  return self.ssr/self.df_resid


In [8]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1, 
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)
two_tailed_new=two_tailed_new[(two_tailed_new["two_tailed"]<=1) & (two_tailed_new["greater"]<=1)]
two_tailed_new["Result_test"]=two_tailed_new["two_tailed"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                  if p_value < alpha else 'Reject005=1 : Homoscedasticity')
#two_tailed_new_show=two_tailed_new.drop(columns="greater")
two_tailed_new.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed,greater,Result_test
Replicate,,,,,,,,,,
2.0,9.0,5.0,6849871.23701396,6348820.721096583,856233.904626745,1587205.1802741457,1.6683346157256473,0.47526897539009516,0.7623655123049524,Reject005=1 : Homoscedasticity
6.0,9.0,5.0,1456272.6861213245,7083203.317298158,182034.08576516557,1770800.8293245395,8.75506770994569,0.005771276846098683,0.9971143615769507,Reject005=0 : Heteroscedasticity
9.0,12.0,2.0,1716494.620639678,5499095.435932638,156044.96551269802,5499095.435932638,19.222065842128824,0.00036245455962813367,0.9998187727201859,Reject005=0 : Heteroscedasticity
10.0,6.0,8.0,646293.4336238493,7140001.150170631,129258.68672476985,1020000.1643100901,8.285711387475814,0.01887983986648778,0.9905600800667561,Reject005=0 : Heteroscedasticity
11.0,7.0,7.0,3540909.5773096345,17604686.545334943,590151.5962182725,2934114.4242224903,4.971797827921631,0.05060846134622099,0.9746957693268895,Reject005=1 : Homoscedasticity
12.0,4.0,10.0,3798656.170049989,34381096.95897994,1266218.7233499964,3820121.884331105,3.6203431339801933,0.2265408671486564,0.8867295664256718,Reject005=1 : Homoscedasticity
13.0,6.0,8.0,1817925.6777812496,10519087.13770727,363585.13555624994,1502726.7339581815,4.339734814081751,0.09034893535917643,0.9548255323204118,Reject005=1 : Homoscedasticity
15.0,6.0,8.0,3743605.610585611,14772381.180339372,748721.1221171222,2110340.1686199103,2.9595227269470303,0.20315983834205054,0.8984200808289747,Reject005=1 : Homoscedasticity
16.0,5.0,9.0,135713.658951358,10159642.81721256,33928.4147378395,1269955.35215157,41.58937319334617,0.0007190063471045338,0.9996404968264477,Reject005=0 : Heteroscedasticity


In [9]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1,
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)

two_tailed_new["two_tailed_new"]=two_tailed_new[["two_tailed","greater"]].apply(lambda value: value[0] if value[0]<0.5 else 2*value[1], axis=1)
two_tailed_new["Result_test"]=two_tailed_new["two_tailed_new"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                      if p_value < alpha else 'Reject005=1 : Homoscedasticity')


two_tailed_new_show=two_tailed_new.drop(columns=["two_tailed", "greater"])
two_tailed_new_show.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed_new,Result_test
Replicate,,,,,,,,,
0.0,10.0,4.0,12178206.85381613,908189.9930129009,1353134.0948684588,302729.99767096696,0.18643754452411707,0.11976055128695659,Reject005=1 : Homoscedasticity
1.0,4.0,10.0,13198303.958854029,587801.4156490258,4399434.652951343,65311.268405447314,0.017814453053407717,1.6308524704297012e-06,Reject005=0 : Heteroscedasticity
2.0,9.0,5.0,6849871.23701396,6348820.721096583,856233.904626745,1587205.1802741457,1.6683346157256473,0.47526897539009516,Reject005=1 : Homoscedasticity
3.0,8.0,6.0,21920611.001485117,807215.3465461938,3131515.8573550167,161443.06930923875,0.049099321577092,0.0016528885817527837,Reject005=0 : Heteroscedasticity
4.0,7.0,7.0,44089183.10692754,1323684.7094380164,7348197.184487923,220614.1182396694,0.030022890336338117,0.00014872603853565303,Reject005=0 : Heteroscedasticity
5.0,9.0,5.0,7857154.277611076,1984307.8138531626,982144.2847013845,496076.95346329064,0.4545862202443179,0.3990091387902706,Reject005=1 : Homoscedasticity
6.0,9.0,5.0,1456272.6861213245,7083203.317298158,182034.08576516557,1770800.8293245395,8.75506770994569,0.005771276846098683,Reject005=0 : Heteroscedasticity
7.0,5.0,9.0,6613025.973296696,8979007.885757796,1653256.493324174,1122375.9857197246,0.7543199942738351,0.6708093625230961,Reject005=1 : Homoscedasticity
8.0,9.0,5.0,20917552.2538173,6842316.615289423,2614694.0317271627,1710579.1538223557,0.5887959431425993,0.5808563718981851,Reject005=1 : Homoscedasticity


In [10]:
two_tailed_new_show['Result_test'].value_counts()

Reject005=1 : Homoscedasticity      51
Reject005=0 : Heteroscedasticity    49
Name: Result_test, dtype: int64